# GPS Interpolation and Mapping 

In this notebook, we will create a map showing motion over a speficied area. In order to accomplish this we will:

1. Load the data table from the GNSS tab of geo-gateway.org

2. Import this data into a pandas dataframe 

3. Use an Ordinary Kriging interpolation to get motion values across a regular grid

4. Plot the interpolated motions on a map 




In [1]:
from create_map import create_map
from gps_interpolation import interpolate, create_grid, reshape_and_create_df
from load_gps_data import load_gps_data
import ipywidgets as widgets
import math

### Loading raw data and Importing data into Pandas data frame##
- This data is loaded with the load_gps_data module created specifically for the type of table that is output by geo-gateway
- Pandas is a python library that creates tables similar to a Microsoft Excel spreadsheet. If you would like to see what the data frame looks like, feel free to type (without quotes),  "gps_df.head()" for a peak at the Dataframe.
- To adapt this to a table you retrieved on geo-gateway, simply change the file name. Be sure it is in the same directory as this notebook. 

In [2]:
uploader = widgets.FileUpload()
uploader
upload_button = widgets.Button(description='Press to upload')
info = widgets.Label()

def on_upload_click(change):
    with open("input_gps.txt", "w+b") as inpt:
        inpt.write(uploader.data[-1])
    global gps_df, deltas
    gps_df = load_gps_data("input_gps.txt")
    deltas = gps_df[['Delta E', 'Delta N', 'Delta V']]
    info.value  = 'uploaded!'


upload_button.on_click(on_upload_click)
VBox3 = widgets.VBox([uploader, upload_button, info])
VBox3

### Interpolating over a regular grid and storing in a new data frame ###
- Here we use an Ordinary Kriging interpolatin to get motion values for each component of motion.
- Different Variogram Models may be selected and give different results. After you run the whole notebook, come back and try a different variogram model. You may choose from the following: linear, gaussian, power, exponential, hole-effect, spherical. To change the model, select the model you want and then press the Do the interpolation button. 
- The maps may be slow to load with very low grid spacing. Typically, a value of .01 degree grid spacing will be sufficient. If you find the maps later in the notebook become slow, try raising the grid spacing. 


In [3]:
interp_button = widgets.Button(description='Do the interpolation')
grid_space = widgets.FloatSlider(value=0.018,
                                 min=0.005,
                                 max=.1,
                                 step=.0005,
                                 description='Grid Spacing: ',
                                 disabled=False,
                                 continous_update=True,
                                 readout=True,
                                 readout_format='.4f',
                                 default=.018)
interpolations = ['linear', 'gaussian', 'power',
                  'exponential', 'hole-effect', 'spherical']
interpolation_type = widgets.RadioButtons(options=interpolations,
                                          description="Interpolation",
                                          default='linear')
azimuth = widgets.IntSlider(value=-5, 
                           min =-180,
                           max=180,
                           step=5,
                           description ='Azimuth', 
                           disabled=False,
                           readout=True,
                           default=-5)
elevation=widgets.IntSlider(value=60, 
                           min=0,
                           max=180,
                           step=1,
                           description='Elevation Angle', 
                           disabled = False,
                           readout=True,
                           default=60)


In [4]:
def on_click_interpolate(change):
    global interpolated_values
    interpolated_values = interpolate(
        gps_df['Lon'],
        gps_df['Lat'],
        grid_spacing=grid_space.value,
        model=interpolation_type.value,
        **deltas)
    losd=to_los_disp(interpolated_values['Delta E'], interpolated_values['Delta N'], interpolated_values['Delta V'], elevation=elevation.value, azimuth=azimuth.value)
    interpolated_values['LOS Displacement'] = losd



def to_los_disp(ux,uy,uv,azimuth=-5, elevation=60):
    g = [math.sin(azimuth)*math.cos(elevation),math.cos(azimuth)*math.cos(elevation),math.sin(elevation)]
    losd = (g[0]*ux+ g[1]*uy+ g[2]*uv)/5.0
    return losd
interp_button.on_click(on_click_interpolate)
VBox1 = widgets.VBox([grid_space, interpolation_type, interp_button, azimuth, elevation])
VBox1


### Mapping the data ##
- Ok now lets throw it on a map! Select the Component you would like to view from the drop down and press Run Interact.
- Change the component of motion in the dropdown menu. It has to redraw the map every time you do this, so be a little patient
- The second map shows the raw values before the interpolation
- You can take the map to fullscreen, hide the minimap, turn on or off the faults and interpolation


In [5]:
motion_menu = widgets.RadioButtons(
    options=['Delta N', 'Delta V', 'Delta E', 'LOS Displacement'],
    value='Delta N',
    description='Component:',
    continuous_update=True,
    layout={'width': 'max-content'}
)

Map_button = widgets.Button(description='Map values')
map_output = widgets.Output()


def on_click_map(change):
    map_output.clear_output()
    my_map = create_map(df=interpolated_values,
                        lon_col='Lon',
                        lat_col='Lat',
                        data_col=motion_menu.value)
    with map_output:
        display(my_map)


Map_button.on_click(on_click_map)
VBox2 = widgets.VBox([motion_menu, Map_button, map_output])
VBox2